## Installing Essential Dependencies

In [1]:
!pip install numpy
!pip install pandas

## Loading Raw Data

In [2]:

# !pip install pandas
import pandas as pd
import numpy as np
df = pd.read_csv('BADSS test data.csv')
df

,Date,Symbol,Maturity,Strike,Bid Price,Bid Size,Ask Price,Ask Size,Undl Price
0,7/31/2024,SPY,8/1/2024,551.0,2.67,2222,2.72,2199,550.81
1,7/31/2024,SPY,8/1/2024,552.0,2.14,1782,2.18,2337,550.81
2,7/31/2024,SPY,8/1/2024,553.0,1.67,1342,1.71,2476,550.81
3,7/31/2024,SPY,8/1/2024,554.0,1.26,1197,1.29,2928,550.81
4,7/31/2024,SPY,8/1/2024,555.0,0.92,1348,0.95,3695,550.81
...,...,...,...,...,...,...,...,...,...
25638,8/30/2024,QQQ,9/13/2024,550.0,0.00,0,0.02,7920,476.27
25639,8/30/2024,QQQ,9/13/2024,555.0,0.00,0,0.02,8946,476.27
25640,8/30/2024,QQQ,9/13/2024,560.0,0.00,0,0.02,9851,476.27
25641,8/30/2024,QQQ,9/13/2024,565.0,0.00,0,0.02,10932,476.27


In [3]:
grouped = df.groupby(['Symbol', 'Maturity', 'Strike'])
result_dict = {}

for (symbol, maturity, strike), group in grouped:
    dates = group['Date'].tolist()
    result_dict[(symbol, maturity, strike)] = dates

for k, v in list(result_dict.items())[500:505]:
    print(f'{k}: {v}')

('IWM', '8/23/2024', 228.0): ['8/9/2024', '8/12/2024', '8/13/2024', '8/14/2024', '8/15/2024', '8/16/2024', '8/19/2024', '8/20/2024', '8/21/2024', '8/22/2024']
('IWM', '8/23/2024', 229.0): ['8/9/2024', '8/12/2024', '8/13/2024', '8/14/2024', '8/15/2024', '8/16/2024', '8/19/2024', '8/20/2024', '8/21/2024', '8/22/2024']
('IWM', '8/23/2024', 230.0): ['8/9/2024', '8/12/2024', '8/13/2024', '8/14/2024', '8/15/2024', '8/16/2024', '8/19/2024', '8/20/2024', '8/21/2024', '8/22/2024']
('IWM', '8/23/2024', 231.0): ['8/9/2024', '8/12/2024', '8/13/2024', '8/14/2024', '8/15/2024', '8/16/2024', '8/19/2024', '8/20/2024', '8/21/2024', '8/22/2024']
('IWM', '8/23/2024', 232.0): ['8/9/2024', '8/12/2024', '8/13/2024', '8/14/2024', '8/15/2024', '8/16/2024', '8/19/2024', '8/20/2024', '8/21/2024', '8/22/2024']


### Missing Data Filling

In [4]:
# Initialize an empty dictionary to store missing dates for each (symbol, maturity, strike) key
missing_dates_dict = {}

# Iterate over each contract identified by (symbol, maturity, strike) and its associated list of dates in result_dict
for (symbol, maturity, strike), dates in result_dict.items():
    if not dates:
        print("Missing!")
    first_date = pd.to_datetime(min(dates))
    maturity_date = pd.to_datetime(maturity) - pd.Timedelta(days=1)
    
    # Create a complete range of dates from the first_date to the maturity_date (as date objects)
    complete_dates = pd.date_range(start=first_date, end=maturity_date).date
    existing_dates = {pd.to_datetime(d).date() for d in dates}
    
    # Determine the missing dates by finding the difference between the complete date range and the existing dates, then sort them
    missing_dates = sorted(set(complete_dates) - existing_dates)
    missing_dates_dict[(symbol, maturity, strike)] = [d.strftime('%Y-%m-%d') for d in missing_dates]

for k, v in list(missing_dates_dict.items())[500:505]:
    print(f'{k}: {v}')


('IWM', '8/23/2024', 228.0): ['2024-08-17', '2024-08-18']
('IWM', '8/23/2024', 229.0): ['2024-08-17', '2024-08-18']
('IWM', '8/23/2024', 230.0): ['2024-08-17', '2024-08-18']
('IWM', '8/23/2024', 231.0): ['2024-08-17', '2024-08-18']
('IWM', '8/23/2024', 232.0): ['2024-08-17', '2024-08-18']


In [5]:
# Convert Date and Maturity columns to datetime
df['Date'] = pd.to_datetime(df['Date'])
df['Maturity'] = pd.to_datetime(df['Maturity'])

# Build new rows for missing dates
new_rows = []
for (symbol, maturity, strike), missing_dates in missing_dates_dict.items():
    for date_str in missing_dates:
        date = date_str
        if date >= "2024-05-12":
            continue
        undl_price = df[(df['Symbol'] == symbol) & (df['Date'] == date)]['Undl Price'].values
        if len(undl_price) == 0:
            continue
        new_row = {
            'Date': date,
            'Symbol': symbol,
            'Maturity': maturity,
            'Strike': strike,
            'Bid Price': 0.0,
            'Bid Size ': 0,
            'Ask Price': 0.0,
            'Ask Size ': 0,
            'Undl Price': undl_price[0]
        }
        new_rows.append(new_row)
        print("New row!")

# Create DataFrame from new rows and combine with original DataFrame
missing_df = pd.DataFrame(new_rows)
combined_df = pd.concat([df, missing_df], ignore_index=True)

# Sort combined DataFrame by Symbol (custom order), Date, Maturity, and Strike
symbol_order = {'SPY': 1, 'IWM': 2, 'QQQ': 3}
combined_df['sort_key'] = combined_df['Symbol'].map(symbol_order)
combined_df.sort_values(by=['sort_key', 'Date', 'Maturity', 'Strike'],
                        ascending=[True, True, True, True],
                        inplace=True)
combined_df.drop(columns=['sort_key'], inplace=True)
combined_df.reset_index(drop=True, inplace=True)

# Format Date and Maturity columns to MM/DD/YYYY and display the first 20 rows
combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce').dt.strftime('%m/%d/%Y')
combined_df['Maturity'] = pd.to_datetime(combined_df['Maturity'], errors='coerce').dt.strftime('%m/%d/%Y')
print(combined_df.head(20))

# Save the final DataFrame to CSV
combined_df.to_csv('BADSS_training_data_filled.csv', index=False)


          Date Symbol    Maturity  Strike  Bid Price  Bid Size   Ask Price  \
0   07/31/2024    SPY  08/01/2024   551.0       2.67       2222       2.72   
1   07/31/2024    SPY  08/01/2024   552.0       2.14       1782       2.18   
2   07/31/2024    SPY  08/01/2024   553.0       1.67       1342       1.71   
3   07/31/2024    SPY  08/01/2024   554.0       1.26       1197       1.29   
4   07/31/2024    SPY  08/01/2024   555.0       0.92       1348       0.95   
5   07/31/2024    SPY  08/01/2024   556.0       0.64       1424       0.66   
6   07/31/2024    SPY  08/01/2024   557.0       0.43       1678       0.45   
7   07/31/2024    SPY  08/01/2024   558.0       0.28       2037       0.29   
8   07/31/2024    SPY  08/01/2024   559.0       0.17       2396       0.18   
9   07/31/2024    SPY  08/01/2024   560.0       0.10       5228       0.11   
10  07/31/2024    SPY  08/01/2024   561.0       0.06       8059       0.07   
11  07/31/2024    SPY  08/01/2024   562.0       0.03       9475 

In [6]:
import pandas as pd
import numpy as np

# Read data
file_path = 'BADSS_training_data_filled.csv'
df = pd.read_csv(file_path)

# Convert "Date" from string to datetime using format "%m/%d/%Y"
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")

# Convert "Date" to numpy datetime64[D] (day precision)
dates_np = df["Date"].values.astype("datetime64[D]")

# Get the earliest date as the reference
start_date_np = dates_np.min()

# Compute trading day number (date_id), skipping weekends
df["date_id"] = np.busday_count(start_date_np, dates_np) + 1

# Display the first few rows
print(df.head())

# Save the updated DataFrame to CSV
df.to_csv("BADSS_training_data_filled.csv", index=False)


        Date Symbol    Maturity  Strike  Bid Price  Bid Size   Ask Price  \
0 2024-07-31    SPY  08/01/2024   551.0       2.67       2222       2.72   
1 2024-07-31    SPY  08/01/2024   552.0       2.14       1782       2.18   
2 2024-07-31    SPY  08/01/2024   553.0       1.67       1342       1.71   
3 2024-07-31    SPY  08/01/2024   554.0       1.26       1197       1.29   
4 2024-07-31    SPY  08/01/2024   555.0       0.92       1348       0.95   

   Ask Size   Undl Price  date_id  
0       2199      550.81        1  
1       2337      550.81        1  
2       2476      550.81        1  
3       2928      550.81        1  
4       3695      550.81        1  


In [7]:
import pandas as pd
import numpy as np

# Read data from CSV
file_path = 'BADSS_training_data_filled.csv'
df = pd.read_csv(file_path)

# Convert "Date" to datetime (auto-infer format)
df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)

# Convert "Date" to numpy datetime64[D] (day precision)
dates_np = df["Date"].values.astype("datetime64[D]")

# Get the earliest date
start_date_np = dates_np.min()

# Calculate trading day numbers (skipping weekends)
df["date_id"] = np.busday_count(start_date_np, dates_np) + 1

# Display the first few rows
print(df.head())

# Save the updated DataFrame to CSV
df.to_csv("BADSS_training_data_filled.csv", index=False)


        Date Symbol    Maturity  Strike  Bid Price  Bid Size   Ask Price  \
0 2024-07-31    SPY  08/01/2024   551.0       2.67       2222       2.72   
1 2024-07-31    SPY  08/01/2024   552.0       2.14       1782       2.18   
2 2024-07-31    SPY  08/01/2024   553.0       1.67       1342       1.71   
3 2024-07-31    SPY  08/01/2024   554.0       1.26       1197       1.29   
4 2024-07-31    SPY  08/01/2024   555.0       0.92       1348       0.95   

   Ask Size   Undl Price  date_id  
0       2199      550.81        1  
1       2337      550.81        1  
2       2476      550.81        1  
3       2928      550.81        1  
4       3695      550.81        1  


/var/folders/5g/05qctxzj46x4ldp4fk1m78lc0000gn/T/ipykernel_86198/442854766.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)


## Calculating PnL \& Exposure

In [8]:
import pandas as pd
import numpy as np

# 1) Read data
file_path = 'BADSS_training_data_filled.csv'
df = pd.read_csv(file_path)

# 2) Convert Date and Maturity columns to datetime
df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)
df["Maturity"] = pd.to_datetime(df["Maturity"], infer_datetime_format=True)

# 3) Compute date_id and expire_date_id (skipping weekends)
dates_np = df["Date"].values.astype("datetime64[D]")
start_date_np = dates_np.min()
df["date_id"] = np.busday_count(start_date_np, dates_np) + 1

maturity_np = df["Maturity"].values.astype("datetime64[D]")
df["expire_date_id"] = np.busday_count(start_date_np, maturity_np) + 1

# 4) Get the maximum trading day number for creating PnL columns
max_date_id = int(df["date_id"].max())

# 5) Build a dictionary: (Symbol, Maturity, Strike, date_id) -> Bid Price
bid_dict = {}
for idx, row in df.iterrows():
    key = (row["Symbol"], row["Maturity"], row["Strike"], row["date_id"])
    bid_dict[key] = row["Bid Price"]

# 6) Add PnL columns (PnL_1 to PnL_max) initialized to 0
for d in range(1, max_date_id + 1):
    df[f"PnL_{d}"] = 0.0

# 7) Fill in PnL for each row
# For a row with date_id = start_d and expire_date_id = end_d,
# for d in [start_d, end_d-1], set PnL_d = Ask Price - Bid Price (of the same contract on the next day)
for idx, row in df.iterrows():
    symbol = row["Symbol"]
    maturity = row["Maturity"]
    strike = row["Strike"]
    ask_price = row["Ask Price"]
    start_d = int(row["date_id"])
    end_d = int(row["expire_date_id"])  # Excluding expiry day itself

    for d in range(start_d, end_d):
        next_key = (symbol, maturity, strike, d + 1)
        if next_key in bid_dict:
            df.at[idx, f"PnL_{d}"] = bid_dict[next_key] - ask_price
        else:
            df.at[idx, f"PnL_{d}"] = 0.0
    df.at[idx, f"PnL_{end_d - 1}"] = -ask_price

# 8) Save the results and display the first 30 rows
df.to_csv("BADSS_training_data_filled.csv", index=False)
print(df.head(30))


/var/folders/5g/05qctxzj46x4ldp4fk1m78lc0000gn/T/ipykernel_86198/3204847741.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)
/var/folders/5g/05qctxzj46x4ldp4fk1m78lc0000gn/T/ipykernel_86198/3204847741.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Maturity"] = pd.to_datetime(df["Maturity"], infer_datetime_format=True)


         Date Symbol   Maturity  Strike  Bid Price  Bid Size   Ask Price  \
0  2024-07-31    SPY 2024-08-01   551.0       2.67       2222       2.72   
1  2024-07-31    SPY 2024-08-01   552.0       2.14       1782       2.18   
2  2024-07-31    SPY 2024-08-01   553.0       1.67       1342       1.71   
3  2024-07-31    SPY 2024-08-01   554.0       1.26       1197       1.29   
4  2024-07-31    SPY 2024-08-01   555.0       0.92       1348       0.95   
5  2024-07-31    SPY 2024-08-01   556.0       0.64       1424       0.66   
6  2024-07-31    SPY 2024-08-01   557.0       0.43       1678       0.45   
7  2024-07-31    SPY 2024-08-01   558.0       0.28       2037       0.29   
8  2024-07-31    SPY 2024-08-01   559.0       0.17       2396       0.18   
9  2024-07-31    SPY 2024-08-01   560.0       0.10       5228       0.11   
10 2024-07-31    SPY 2024-08-01   561.0       0.06       8059       0.07   
11 2024-07-31    SPY 2024-08-01   562.0       0.03       9475       0.04   
12 2024-07-3

In [9]:
import pandas as pd
import numpy as np

# Step 1: Read data
file_path = 'BADSS_training_data_filled.csv'
df = pd.read_csv(file_path)

# Step 2: Convert "Date" and "Maturity" to datetime
df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)
df["Maturity"] = pd.to_datetime(df["Maturity"], infer_datetime_format=True)

# Step 3: Compute date_id and expire_date_id (skipping weekends)
dates_np = df["Date"].values.astype("datetime64[D]")
start_date_np = dates_np.min()
df["date_id"] = np.busday_count(start_date_np, dates_np) + 1

maturity_np = df["Maturity"].values.astype("datetime64[D]")
df["expire_date_id"] = np.busday_count(start_date_np, maturity_np) + 1

# Step 4: Determine max trading day number for exposure columns
max_date_id = int(df["date_id"].max())

# Step 5: Add exposure columns (exposure_1 to exposure_max) initialized to 0
for d in range(1, max_date_id + 1):
    df[f"exposure_{d}"] = 0.0

# Step 6: Build a lookup dictionary: (Symbol, Strike, date_id) -> Undl Price
underlying_dict = {}
for idx, row in df.iterrows():
    key = (row["Symbol"], row["Strike"], int(row["date_id"]))
    underlying_dict[key] = row["Undl Price"]

# Step 7: Define ratio mapping by Symbol
ratio_dict = {"SPY": 1.03, "IWM": 1.036, "QQQ": 1.041}

# Step 8: Calculate exposure for each row in its effective period (from date_id to expire_date_id - 1)
for idx, row in df.iterrows():
    symbol = row["Symbol"]
    strike = row["Strike"]
    ratio = ratio_dict.get(symbol, 1.0)  # Default ratio is 1.0 if symbol not found
    start_d = int(row["date_id"])
    end_d = int(row["expire_date_id"])
    
    for d in range(start_d, end_d):
        key = (symbol, strike, d)
        undl_price = underlying_dict.get(key, 0.0)
        exposure_val = max(undl_price * ratio - strike, 0) - max(undl_price - strike, 0)
        df.at[idx, f"exposure_{d}"] = exposure_val

# Step 9: Save results and display the first 30 rows
df.to_csv("BADSS_training_data_filled.csv", index=False)
print(df.head(30))


/var/folders/5g/05qctxzj46x4ldp4fk1m78lc0000gn/T/ipykernel_86198/1631010626.py:9: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)
/var/folders/5g/05qctxzj46x4ldp4fk1m78lc0000gn/T/ipykernel_86198/1631010626.py:10: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df["Maturity"] = pd.to_datetime(df["Maturity"], infer_datetime_format=True)


         Date Symbol   Maturity  Strike  Bid Price  Bid Size   Ask Price  \
0  2024-07-31    SPY 2024-08-01   551.0       2.67       2222       2.72   
1  2024-07-31    SPY 2024-08-01   552.0       2.14       1782       2.18   
2  2024-07-31    SPY 2024-08-01   553.0       1.67       1342       1.71   
3  2024-07-31    SPY 2024-08-01   554.0       1.26       1197       1.29   
4  2024-07-31    SPY 2024-08-01   555.0       0.92       1348       0.95   
5  2024-07-31    SPY 2024-08-01   556.0       0.64       1424       0.66   
6  2024-07-31    SPY 2024-08-01   557.0       0.43       1678       0.45   
7  2024-07-31    SPY 2024-08-01   558.0       0.28       2037       0.29   
8  2024-07-31    SPY 2024-08-01   559.0       0.17       2396       0.18   
9  2024-07-31    SPY 2024-08-01   560.0       0.10       5228       0.11   
10 2024-07-31    SPY 2024-08-01   561.0       0.06       8059       0.07   
11 2024-07-31    SPY 2024-08-01   562.0       0.03       9475       0.04   
12 2024-07-3

## Spliting Training Data and Testing Data

In [11]:
t1, t2 = 1, 22  # Training set range based on date_id
t3, t4 = 1, 23  # Testing set range based on date_id

# Filter data by date_id for training and testing
train_data = df[(df["date_id"] >= t1) & (df["date_id"] <= t2)]
test_data = df[(df["date_id"] >= t3) & (df["date_id"] <= t4)]

# Save the filtered data to new CSV files with the same format as the original
train_data.to_csv("BADSS_testing_data_1.csv", index=False)
test_data.to_csv("BADSS_testing_data_2.csv", index=False)
